In [1]:
from math import *
import numpy as np
import pandas as pd
import datetime as dt
from pytz import timezone
import json

import skyfield
from skyfield.api import load
from skyfield.api import N, W, S, E, wgs84
from skyfield.api import Star
from skyfield.data import hipparcos
from skyfield.almanac import find_discrete, risings_and_settings
from skyfield.magnitudelib import planetary_magnitude
from skyfield import almanac

In [2]:
ts = load.timescale()
ephemeris = load('de421.bsp')

sun = ephemeris['SUN']
earth = ephemeris['EARTH']
moon = ephemeris['MOON']

with load.open(hipparcos.URL) as f:
    stars_df = hipparcos.load_dataframe(f)

planet_dictionary = {"Venus":"VENUS", "Mars":"MARS", "Jupiter":"JUPITER BARYCENTER", "Saturn":"SATURN BARYCENTER"}
    
star_dictionary = {"Alpheratz":677, "Ankaa":2081, "Schedar":3179, "Diphda":3419, "Achernar":7588, "Hamal":9884, "Polaris":11767, "Acamar":13847, "Menkar":14135, "Mirfak":15863, "Aldebaran":21421, "Rigel":24436, "Capella":24608, "Bellatrix":25336, "Elnath":25428, "Alnilam":26311, "Betelgeuse":27989, "Canopus":30438, "Sirius":32349, "Adhara":33579, "Procyon":37279, "Pollux":37826, "Avior":41037, "Suhail":44816, "Miaplacidus":45238, "Alphard":46390, "Regulus":49669, "Dubhe":54061, "Denebola":57632, "Gienah":59803, "Acrux":60718, "Gacrux":61084, "Alioth":62956, "Spica":65474, "Alkaid":67301, "Hadar":68702, "Menkent":68933, "Arcturus":69673, "Rigil Kent.":71683, "Kochab":72607, "Zuben'ubi":72622, "Alphecca":76267, "Antares":80763, "Atria":82273, "Sabik":84012, "Shaula":85927, "Rasalhague":86032, "Eltanin":87833, "Kaus Aust.":90185, "Vega":91262, "Nunki":92855, "Altair":97649, "Peacock":100751, "Deneb":102098, "Enif":107315, "Al Na'ir":109268, "Fomalhaut":113368, "Scheat":113881, "Markab":113963}

In [3]:
def createAngle(degrees, minutes, sign):
    if (sign == None): 
        sign = 1
    return (degrees + minutes/60.0) * sign

def decompose_int(value, places=0):
    """Decompose `value` into units, minutes, seconds, and second fractions.
    This routine prepares a value for sexagesimal display, with its
    seconds fraction expressed as an integer with `places` digits.  The
    result is a tuple of five integers:
    ``(sign [either +1 or -1], units, minutes, seconds, second_fractions)``
    The integers are properly rounded per astronomical convention so
    that, for example, given ``places=3`` the result tuple ``(1, 11, 22,
    33, 444)`` means that the input was closer to 11u 22' 33.444" than
    to either 33.443" or 33.445" in its value.
    """
    power = 10 ** places
    n = int((power * 3600 * value + 0.5) // 1.0)
    sign = np.sign(n)
    n, fraction = divmod(abs(n), power)
    n, seconds = divmod(n, 60)
    n, minutes = divmod(n, 60)
    return sign, n, minutes, seconds, fraction

def decompose_float(value, places=0):
    """Decompose `value` into units, minutes, and seconds.
    Note that this routine is not appropriate for displaying a value,
    because rounding to the smallest digit of display is necessary
    before showing a value to the user.  Use `_sexagesimalize_to_int()`
    for data being displayed to the user.
    This routine simply decomposes the floating point `value` into a
    sign (+1.0 or -1.0), units, minutes, and seconds, returning the
    result in a four-element tuple.
    >>> _sexagesimalize_to_float(12.05125)
    (1.0, 12.0, 3.0, 4.5)
    >>> _sexagesimalize_to_float(-12.05125)
    (-1.0, 12.0, 3.0, 4.5)
    """
    sign = np.sign(value)
    n = abs(value)
    minutes, seconds = divmod(n * 3600.0, 60.0)
    units, minutes = divmod(minutes, 60.0)
    return sign, units, minutes, seconds

In [4]:
class CelestialObject(json.JSONEncoder):
    def __init__(self, name, objtype, magnitude, ra, dec, distance):
        self.name = name
        self.objtype = objtype
        self.magnitude = magnitude
        self.ra = ra
        self.dec = dec
        self.distance = distance

In [5]:
def getCelestialObjectData(lat, lon, rise_start, rise_end):
    objects = []
    
    # The location and time for which we'll fetch star data
    earth_at = (earth + wgs84.latlon(lat, lon)).at(rise_start) 
    
    # Sun
    radec = earth_at.observe(sun).apparent().radec()
    objects.append(CelestialObject("Sun", "Sun", 0.0, radec[0].hours, radec[1].degrees, radec[2].au))
    
    # Moon
    radec = earth_at.observe(moon).apparent().radec()
    objects.append(CelestialObject("Moon", "Moon", 0.0, radec[0].hours, radec[1].degrees, radec[2].au))
    
    # Planets
    for planet in planet_dictionary:
        astrometric = earth_at.observe(ephemeris[planet_dictionary[planet]])
        radec = astrometric.apparent().radec()
        magnitude = planetary_magnitude(astrometric)
        objects.append(CelestialObject(planet, "Planet", float(magnitude), radec[0].hours, radec[1].degrees, radec[2].au))
    
    # Stars
    for star_name in star_dictionary:
        # Load dataframe separately to get magnitude
        star_df = stars_df.loc[star_dictionary[star_name]]
        
        # Load star object
        star = Star.from_dataframe(star_df)
        
        # Get apparent RA and dec for given lat/lon
        radec = earth_at.observe(star).apparent().radec()        
        objects.append(CelestialObject(star_name, "Star", star_df.magnitude, radec[0].hours, radec[1].degrees, radec[2].au))

        
    return objects

In [6]:
lat = createAngle(19, 42.47, N)
lon = createAngle(155, 58.73, W)
rise_start = ts.utc(2022, 12, 8)
rise_end = None

data = getCelestialObjectData(lat, lon, rise_start, rise_end)

In [7]:
json_string = json.dumps([ob.__dict__ for ob in data])

In [8]:
json_string

'[{"name": "Sun", "objtype": "Sun", "magnitude": 0.0, "ra": 16.95325160239414, "dec": -22.653099491524365, "distance": 0.9851023678339856}, {"name": "Moon", "objtype": "Moon", "magnitude": 0.0, "ra": 4.82126719969415, "dec": 24.47869455293052, "distance": 0.0026985608809493443}, {"name": "Venus", "objtype": "Planet", "magnitude": -3.9149340890596664, "ra": 17.77875814095296, "dec": -24.0522624720729, "distance": 1.6663381278160545}, {"name": "Mars", "objtype": "Planet", "magnitude": -1.9427681831904258, "ra": 4.95960945749927, "dec": 24.957004979534503, "distance": 0.5492552199303701}, {"name": "Jupiter", "objtype": "Planet", "magnitude": -2.523551465907854, "ra": 23.96491084605625, "dec": -1.7469493539453387, "distance": 4.631670635614837}, {"name": "Saturn", "objtype": "Planet", "magnitude": 0.8091679882863634, "ra": 21.52203999932492, "dec": -16.001012709575445, "distance": 10.225820528367898}, {"name": "Alpheratz", "objtype": "Star", "magnitude": 2.07, "ra": 0.13997365948719964, "d